In [ ]:
from grid.refine import *

In [ ]:
m = jittens.machines.machines()['i-06f75c5863482f338']

In [ ]:
m.fetch('2021-02-15 16-18-01 exotic-swell', 'output/pavlov/*/', 'output/tmp')

In [ ]:
conn.host